# Imports:

In [ ]:
import sys
sys.float_info
#Should reveal that floats have 8 bytes

In [ ]:
MD = "metaData"
Indep = "trainIndependents"
Dep = "trainDependent"
fileNames = [MD, Indep, Dep]

- For Indep: With 23081 columns, each of which store one float (although some store NAs), each row has (high-end estimate) 23081*8 ≈ 185 KB. (Datatypes, NAs, etc. may cause structure to be slightly different)

Using this as a rough benchmark, that means that dealing with 1000 rows deals with 185 MB of data. This can be used to guide how much data we want to work with to start with. Then, we can expand our algorithm to dealing with the entire dataset.

- Dep: 1155 columns ≈ 9.24 KB

I'm kind of afraid to re-open up either Indep or Dep on my computer, so if someone could, I think Numbers will tell you the # of rows, even though it can't show them all. That would be super helpful to estimate running time of algorithm on entire set based on subset, among other things - I can't tell the shape of Dask dataframes for reasons explicated upon below

In [14]:
import math ##For basic mathematical operations
import matplotlib.pyplot as plt
import dask.dataframe as dd  
from IPython import display ## Plot setup for Ipython
from matplotlib import cm ##  Colormap reference
from matplotlib import gridspec ##plot setups
from matplotlib import pyplot as plt ##plot setups
import numpy as np 
import pandas as pd
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
import warnings
# warnings.filterwarnings("ignore") - may want to ignore, if know what you are doing, but can be helpful
# warnings.filterwarnings(action = "once") - may want to show only once
# from Queue import priority_queue

3it [19:09, 383.03s/it]


# Reading the Data in

## Important Note

Please restart the kernel, if trying to read the data again/through an alternate method, to keep memory down, and everything fast - see more notes in my Slack discussion w/ Daisuke


## Can use pandas, but don't because of time it takes 
w/o multithreading: 3it [28:17, 565.71s/it] - 3it refers to # of files


Would probably still take 90% of the time b/c of largest file size


In [ ]:
# frames = []

# for index, fileName in tqdm(enumerate(fileNames)):
#     frames.append(dd.read_csv("../../../Downloads/TestData/" + fileName + ".csv", sep=",", sample = 2000000))

# To experiment with partial number of rows and panda, you can/should try running code below

Please check out all of the cool extra options in pandas.read_csv: might be worth running verbose = True for all rows (not just subset like nrows = 1000), b/c even though it will take about 0.5 hr, it is worth doing for extra information (maybe just leave computer running overnight or open up diff. Jupyter notebook tab): https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

- Below: nrows = 1000 (so just looks at first 1000 rows)
- verbose = True, prints out number of NA values

You can see some of the output below, but had to stop running b/c of time + memory - try with fewer rows maybe?

In [51]:
frames = []

for index, fileName in tqdm(enumerate(fileNames)):
    frames.append(pd.read_csv("../../../Downloads/TestData/" + fileName + ".csv", sep=",", nrows = 1000, verbose = True))

0it [00:00, ?it/s]

Tokenization took: 0.03 ms
Type conversion took: 39.76 ms
Parser memory cleanup took: 0.01 ms


1it [00:00,  3.00it/s]

Tokenization took: 137.33 ms
Type conversion took: 27002.56 ms
Parser memory cleanup took: 20.25 ms
Tokenization took: 71.12 ms
Type conversion took: 22350.52 ms
Parser memory cleanup took: 26.58 ms
Tokenization took: 77.90 ms
Type conversion took: 26514.34 ms
Parser memory cleanup took: 32.13 ms
Tokenization took: 55.99 ms
Type conversion took: 31076.12 ms
Parser memory cleanup took: 22.23 ms
Tokenization took: 83.54 ms
Type conversion took: 29207.98 ms
Parser memory cleanup took: 19.94 ms
Tokenization took: 60.45 ms
Type conversion took: 30361.69 ms
Parser memory cleanup took: 2.06 ms
Tokenization took: 51.66 ms
Type conversion took: 27530.16 ms
Parser memory cleanup took: 5.41 ms
Tokenization took: 68.81 ms
Type conversion took: 38961.70 ms
Parser memory cleanup took: 2.85 ms
Tokenization took: 52.58 ms
Type conversion took: 18128.72 ms
Parser memory cleanup took: 0.55 ms
Tokenization took: 27.90 ms
Type conversion took: 18731.04 ms
Parser memory cleanup took: 0.54 ms
Tokenization t

distributed.worker - WARNING - Worker is at 64% memory usage. Resuming worker. Process memory: 1.28 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Worker is at 250% memory usage. Pausing worker.  Process memory: 5.00 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 4.90 GB -- Worker memory limit: 2.00 GB
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-51-29250af15388>", line 4, in <module>
    frames.append(pd.read_csv("../../../Downloads/TestData/" + fileName + ".csv", sep=",", nrows = 1000, verbose = True))
  File "/opt/anaconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 685, in parser_f
    return _read(filepath_or_buffer, kwds)
  File "/opt/anaconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 463, in _read
    data = parser.read(nrows)
  File "/opt/anaconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 1154, in read
    ret = self._engine.read(nrows)
  File "/opt/anaconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 2059, in read
    data = self._reader.read(nrows)
  File "pandas/_libs/parsers.pyx", line 881, in pandas._libs.parsers.TextReader.read
  File "pandas

KeyboardInterrupt: 

# Can use dask (resources for learning at end of this notebook) maybe we can make a workbook to each other this once we figure out different aspects of how it works?

## Dask Documentation

https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.Series.unique

Reading files is really quick: 3it [02:11, 43.81s/it] on my last run, can take as high as ~3 min (for all 3 files together)

Dask is fast b/c it uses multiple dataframes under the hood instead of 1. 

    https://docs.dask.org/en/latest/dataframe.html
    
## Reading quickly!

Need to read advice at bottom, especially for parquet: https://examples.dask.org/dataframes/01-data-access.html
    
## Scheduling Tasks

In addition, it also "schedules tasks" for later, whenever you ask it to do something - I don't think the entire thing would actually print if you tried to print it after reading the CSV, because the task of fully storing the read values in the correct slot has not been done yet. If you print out dataframe.head(), you get NAs at the top, but if you try other things like printing all the unique values in a column, it will only have float_64 and ... printed out and other things indicating that the task has been "delayed".

    frames[-1] = frames[-1].fillna(frames[-1].mean)
=> This is the command that ideally in the for loop after reading in the CSV file would fill in the correct values, but with pandas it takes FOREVER and with dask, it works quickly, but only because tasks are being scheduled, but not actually executed.

- What are delayed tasks and do they get delayed automatically?

https://docs.dask.org/en/latest/delayed.html

- Sometimes, in the statements below, you can see that it seems there are 7 tasks and 27 dataframes with the Indep DF


# Problems

- Trying to use the compute command does not work. For example, trying 
    
    frame[2].compute()
(computing the Dep file, which is < 1GB) took
    
    1:38:29 <00:00, 5909.91s/it
and still didn't work!!!

- What does it mean to visualize tasks exactly? Does it only work after .compute()?

- What is sample?

Have to set sample to a high number, otherwise another warning is printed out; not quite sure what it means (maybe describes size of the CSV)


## Keep getting this warning, not sure what it means exactly - often found when reading/computing:

    distributed.worker WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.40 GB -- Worker memory limit: 2.00 GB

You will get a lot of notifications from code below about high memory usage, possible data leakage, etc. (not sure if it’s just complaining b/c it’s so memory-intensive or if my code may not be completely right)

I thought it referred to table size, but it went up and down - I feel like this is because the memory limit is being reached, data is being cleared, and then the process restarts


In [20]:
frames = []

for index, fileName in tqdm(enumerate(fileNames)):
    frames.append(dd.read_csv("../../../Downloads/TestData/" + fileName + ".csv", sep=",", sample = 2000000))

3it [02:11, 43.81s/it]


ALSO can try getting subsets with [:1000, :] - first 1000 rows, and all columns

Try playing around with this, .iloc, .loc, or slicing later to see what is faster

Disadvantages with this - reads in entire csv THEN creates subset

Not sure if nrows or even skiprows can be used like above with pandas

In [ ]:
frames = []

for index, fileName in tqdm(enumerate(fileNames)):
    frames.append(dd.read_csv("../../../Downloads/TestData/" + fileName + ".csv", sep=",", sample = 2000000)[:1000, :])

# Speed

As Jupyter Notebook has limited resources in memory, etc., at least by default, and anyway adds ~20% overhead to run time, run final code in script.


## To time something

For a for loop, to see iterations as they happen, wrap collection in tqdm (already imported above). 

tqdm is a library that essentially shows you the number of iterations in a for loop
So it can be useful when you are doing 10,000 iterations and want to see the overall time it has taken (2:48) and average speed (56.14 s/ it). It also shows you your current speed while running (e.g. how many seconds your last few iterations took on average)

For a non-for loop, can do timeit (this will suppress output that is returned from being shown) or can wrap in for loop like 
    
    for i in range(1):


# Measuring size of a Dask DataFrame

Can use len(df.columns), but len(df) will take LONG time to run

Same for df.count() and df.index

Try this: https://stackoverflow.com/questions/50355598/how-should-i-get-the-shape-of-a-dask-dataframe

Pandas command don't completely work over here b/c of scheduled tasks, so this is not as helpful: https://stackoverflow.com/questions/15943769/how-do-i-get-the-row-count-of-a-pandas-dataframe

df.shape works, but has scheduled tasks next to # of columns, so still doesn't show number of rows - can someone include this info

In [26]:
frames[1].shape

(Delayed('int-2a79f10f-dfc7-4ae9-b188-6766fb9a4e4f'), 23081)

In [33]:
timeit frames[1].shape

557 µs ± 76.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [37]:
(frames[1]).count()

Dask Series Structure:
npartitions=1
Symbol 0000 Independent 00    int64
Time                            ...
dtype: int64
Dask Name: dataframe-count-agg, 581 tasks

In [6]:
frames[-1].count

<bound method _Frame.count of Dask DataFrame Structure:
                 Time Symbol 0000 Dependent Symbol 0001 Dependent Symbol 0002 Dependent Symbol 0003 Dependent Symbol 0004 Dependent Symbol 0005 Dependent Symbol 0006 Dependent Symbol 0007 Dependent Symbol 0008 Dependent Symbol 0009 Dependent Symbol 0010 Dependent Symbol 0011 Dependent Symbol 0012 Dependent Symbol 0013 Dependent Symbol 0014 Dependent Symbol 0015 Dependent Symbol 0016 Dependent Symbol 0017 Dependent Symbol 0018 Dependent Symbol 0019 Dependent Symbol 0020 Dependent Symbol 0021 Dependent Symbol 0022 Dependent Symbol 0023 Dependent Symbol 0024 Dependent Symbol 0025 Dependent Symbol 0026 Dependent Symbol 0027 Dependent Symbol 0028 Dependent Symbol 0029 Dependent Symbol 0030 Dependent Symbol 0031 Dependent Symbol 0032 Dependent Symbol 0033 Dependent Symbol 0034 Dependent Symbol 0035 Dependent Symbol 0036 Dependent Symbol 0037 Dependent Symbol 0038 Dependent Symbol 0039 Dependent Symbol 0040 Dependent Symbol 0041 Dependen

In [29]:
timeit frames[2].shape

13.9 ms ± 2.16 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [42]:
frames[0].memory_usage(deep=True) # No idea what this does - this might have accomplished something in pandas

Dask Series Structure:
npartitions=1
    int64
      ...
dtype: int64
Dask Name: series-groupby-sum-agg, 7 tasks

In [50]:
len(frames[2].columns)

1155

This was running for a few dozen minutes and did not work

In [49]:
frames[1].describe().compute()  


distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.40 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.40 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.41 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.41 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.42 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARN

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.59 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.60 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - gc.collect() took 5.941s. This is usually a sign that some tasks handle too many Python objects at the same time. Rechunking the work into smaller tasks might help.
distributed.worker - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 1.60 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.60 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.75 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.76 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.77 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.77 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.78 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARN

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.96 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.97 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.98 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.98 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.99 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARN

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.19 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.19 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.19 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.20 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.20 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARN

...

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.75 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.72 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.65 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.62 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.58 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARN

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.03 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.00 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.97 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.95 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.94 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARN

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.66 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.63 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.63 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.64 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.65 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARN

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 5.11 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.54 GB -- Worker memory limit: 2.00 GB


KeyboardInterrupt: 

# Visualize

After compute, you can look at network of schedules with visualize, although since compute doesn't work, visualize doesn't

Maybe try it with a smaller subset of the original dataframes?

In [19]:
import graphviz 

frames[1].visualize()

ExecutableNotFound: failed to execute ['dot', '-Tpng'], make sure the Graphviz executables are on your systems' PATH

Can print just top few lines with head command to see it...

In [3]:
frames[-1].head()

,Time,Symbol 0000 Dependent,Symbol 0001 Dependent,Symbol 0002 Dependent,Symbol 0003 Dependent,Symbol 0004 Dependent,Symbol 0005 Dependent,Symbol 0006 Dependent,Symbol 0007 Dependent,Symbol 0008 Dependent,...,Symbol 1144 Dependent,Symbol 1145 Dependent,Symbol 1146 Dependent,Symbol 1147 Dependent,Symbol 1148 Dependent,Symbol 1149 Dependent,Symbol 1150 Dependent,Symbol 1151 Dependent,Symbol 1152 Dependent,Symbol 1153 Dependent
0,20070103_093000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20070103_100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20070103_103000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20070103_110000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20070103_113000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Faster ways to store data

## - Apache Hadoop
## - columnar parquet

In [9]:
frames[-1].compute()

,Time,Symbol 0000 Dependent,Symbol 0001 Dependent,Symbol 0002 Dependent,Symbol 0003 Dependent,Symbol 0004 Dependent,Symbol 0005 Dependent,Symbol 0006 Dependent,Symbol 0007 Dependent,Symbol 0008 Dependent,...,Symbol 1144 Dependent,Symbol 1145 Dependent,Symbol 1146 Dependent,Symbol 1147 Dependent,Symbol 1148 Dependent,Symbol 1149 Dependent,Symbol 1150 Dependent,Symbol 1151 Dependent,Symbol 1152 Dependent,Symbol 1153 Dependent
0,20070103_093000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20070103_100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20070103_103000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20070103_110000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20070103_113000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457,20191129_103000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1458,20191129_110000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1459,20191129_113000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,20191129_120000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
frames[2].head()

,Time,Symbol 0000 Dependent,Symbol 0001 Dependent,Symbol 0002 Dependent,Symbol 0003 Dependent,Symbol 0004 Dependent,Symbol 0005 Dependent,Symbol 0006 Dependent,Symbol 0007 Dependent,Symbol 0008 Dependent,...,Symbol 1144 Dependent,Symbol 1145 Dependent,Symbol 1146 Dependent,Symbol 1147 Dependent,Symbol 1148 Dependent,Symbol 1149 Dependent,Symbol 1150 Dependent,Symbol 1151 Dependent,Symbol 1152 Dependent,Symbol 1153 Dependent
0,20070103_093000,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...
1,20070103_100000,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...
2,20070103_103000,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...
3,20070103_110000,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask DataFrame St...,<bound method _Frame.mean of Dask 

In [4]:
for i in tqdm(range(1)):
    frames[2].compute()

100%|██████████| 1/1 [1:38:29<00:00, 5909.91s/it]


In [5]:
for i in tqdm(range(1)):
    frames[2].head()

100%|██████████| 1/1 [06:23<00:00, 383.39s/it]


In [38]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')


In [39]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://192.168.1.64/1601/1 Dashboard: http://192.168.1.64/1601/1:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


In [40]:
HTML('<iframe width="1000" height="1000" src="https://examples.dask.org/dataframes/01-data-access.html" frameborder="0" allowfullscreen></iframe>')
# check out https://examples.dask.org/dataframes/01-data-access.html